In [2]:
from utils import db_info

import re
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

---

### 문제
- 제공되는 clinical note(한 환자의 의료 기록 샘플) 로부터 정보를 추출하여 4개의 테이블로 세분화 후 DB에 저장
- 테이블 : person, visit_occurrence, condition_occurrence, drug_exposure


### 조건
- 제공된 note에서 "CONTINUING" 제외
- 환자 id 및 각 테이블의 id는 랜덤하게 중복없이 부여

---

#### 데이터베이스 접속
- data.ini를 사용해 hide variable 진행

In [3]:
connection_format = f'postgresql://{db_info["USER"]}:{db_info["PASSWORD"]}@{db_info["HOST"]}:{db_info["PORT"]}/{db_info["DB"]}'
        
client = create_engine(connection_format)

#### Clinical Note에 대한 데이터 로드
- 개수는 총 23개 이다.

In [4]:
note_df = pd.read_sql('select * from de.clinical_note;', con=client)
print(len(note_df))

23


#### Clinical Note Sample - 1
- 첫번째 진단 기록처럼 방문, 진단, 처방 모두 한번씩 이루어질 수 있음

In [13]:
print(note_df.iloc[0]['note'])

Andrea7 Wolf938
Race:                White
Ethnicity:           Non-Hispanic
Gender:              M
Age:                 55
Birth Date:          1965-04-22
Marital Status:      M
--------------------------------------------------------------------------------
ALLERGIES:
No Known Allergies
--------------------------------------------------------------------------------
ENCOUNTER
2011-06-20 : Encounter at Cape Cod Vet Center : Encounter for Acute bronchitis (disorder)
Type: ambulatory
   
   MEDICATIONS:
  2011-06-20 : Acetaminophen 325 MG Oral Tablet for Acute bronchitis (disorder)
   
   CONDITIONS:
  2011-06-20 : Acute bronchitis (disorder)
   
   CARE PLANS:
  2011-06-20 : Respiratory therapy
                         Reason: Acute bronchitis (disorder)
                         Activity: Recommendation to avoid exercise
                         Activity: Deep breathing and coughing exercises
   
   REPORTS:
   
   OBSERVATIONS:
   
   PROCEDURES:
  2011-06-20 : Sputum examination (pro

#### Clinical Note Sample - 2
- 두번째 진단 기록처럼 방문하고 여러 진단을 받았지만 처방은 이루어지지 않는 경우도 존재

In [12]:
print(note_df.iloc[22]['note'])

Andrea7 Wolf938
Race:                White
Ethnicity:           Non-Hispanic
Gender:              M
Age:                 55
Birth Date:          1965-04-22
Marital Status:      M
--------------------------------------------------------------------------------
ALLERGIES:
No Known Allergies
--------------------------------------------------------------------------------
ENCOUNTER
2020-02-25 : Encounter at Cape Cod Vet Center
Type: ambulatory
   
   MEDICATIONS:
   
   CONDITIONS:
  2020-02-25 : COVID-19
  2020-02-25 : Suspected COVID-19
  2020-02-25 : Loss of taste (finding)
  2020-02-25 : Fever (finding)
  2020-02-25 : Diarrhea symptom (finding)
  2020-02-25 : Wheezing (finding)
  2020-02-25 : Dyspnea (finding)
  2020-02-25 : Sore throat symptom (finding)
  2020-02-25 : Cough (finding)
   
   CARE PLANS:
  2020-02-25 : Infectious disease care plan (record artifact)
                         Reason: COVID-19
                         Activity: Airborne precautions (procedure)
             

---

#### Note에서 person 정보를 추출하는 함수
- person 정보는 note마다 고유한 값이기에 반복을 통해 추출할 필요가 없음

<br/>

- person_num : note 수만큼의 범위에서 비복원추출
- birth_date : `\s+?` > `\s*`로 변경, 0000-00-00의 포맷에 대한 문자열을 갖고옴
    - year_of_birth : birth_date에서 0000만 추출
    - month_of_birth : birth_date에서 중간의 00만 추출
    - day_of_birth : birth_date에서 마지막의 00만 추출
- death_date : note 정보에서 찾지 못했음
- gender_value, race_value, ethnicity_value 해당 포맷에 맞게 추출

In [4]:
def input_person(text):
    person_data = {}
    
    # person_id 중복여부 검사
    person_num = np.random.choice(len(note_df), 1)
    while person_num in person_df['person_id'].values:
        person_num = np.random.choice(len(note_df), 1)

    person_data['person_id'] = int(person_num)
    
    # birth_date 추출 후 split을 사용한 년, 월, 일 구분
    birth_date = re.findall('Birth Date:\s+?([0-9]{4}-[0-9]{2}-[0-9]{2})', text)[0]
    person_data['year_of_birth'] = birth_date.split('-')[0]
    person_data['month_of_birth'] = birth_date.split('-')[1]
    person_data['day_of_birth'] = birth_date.split('-')[2]
    
    # 성별, 인종, 민족성에 대한 정보 추출
    person_data['gender_value'] = re.findall('Gender:\s*(\w+)', text)[0]
    person_data['race_value'] = re.findall('Race:\s*(\w+)', text)[0]
    person_data['ethnicity_value'] = re.findall('Ethnicity:\s*([a-zA-Z-]+)', text)[0]
    
    # 해당 note에서 추출한 person의 정보 저장
    person_df.loc[len(person_df)] = person_data
    
    return person_num

#### Note에서 visit_occurrence 정보를 추출하는 함수
- visit_occurrence 정보를 추출할 수 있는 텍스트 범위만 추출
- visit_occurrence 정보는 note마다 고유한 정보기에 반복을 통해 추출할 필요 없음

<br/>

- visit_num : 범위를 1000 ~ 1999 로 정해 visit_num 만의 범위 설정 > 해당 범위에서 비복원추출 진행
- person_id : foreign_key인 person_id를 인자로 받아 추출
- visit_start_date : ENCOUNTER 뒤의 0000-00-00 포맷에 대한 문자열 추출
- care_site_nm : Encounter at 뒤의 공백포함 대소문자 구분없는 문자열 추출
- visit_type_value : Type: 뒤의 공백포함 대소문자 구분없는 문자열 추출

In [5]:
def input_visit_occurrence(text, person_id):
    # ENCOUNTER ~ MEDICATIONS 사이의 텍스트만 추출
    visit_text = text[text.find('ENCOUNTER'):text.find('MEDICATIONS')]
    
    # visit_occurrence_id에 대한 중복여부 검사
    visit_num = random.randint(1000, 1999)
    while visit_num in visit_df['visit_occurrence_id'].values:
        visit_num = random.randint(1000, 1999)
        
    visit_data = {}
    
    # 해당 note의 person_id와 visit_occurrence_id 정보 추출
    visit_data['visit_occurrence_id'] = int(visit_num)
    visit_data['person_id'] = int(person_id)
    
    # 해당 note의 visit_start_date, care_site_nm, visit_type_value 정보 추출
    visit_data['visit_start_date'] = re.findall('ENCOUNTER\s+([0-9]{4}-[0-9]{2}-[0-9]{2})', visit_text)[0]
    visit_data['care_site_nm'] = re.findall('Encounter at\s+([ a-zA-Z]+)', visit_text)[0]
    visit_data['visit_type_value'] = re.findall('Type:\s+([ a-zA-Z]+)', visit_text)[0]
    
    # 해당 note에서 추출한 visit_occurrence의 정보 저장
    visit_df.loc[len(visit_df)] = visit_data
    
    return visit_num

#### Note에서 drug_exposure 정보를 추출하는 함수
- drug_exposure 정보를 추출할 수 있는 텍스트 범위만 추출
- 한 note 당 여러 약에 대한 처방이 이루어질 수도 있기에 반복문을 사용한 정보 추출
- 방문 및 진단만 한 note도 있기에 예외처리를 통해 data 추출

<br/>

1. 약 처방의 포맷이 처음에 date 문자열부터 시작하기에 약처방리스트에 0000-00-00 포맷의 문자열을 추출 후 저장
2. 약처방리스트를 반복문을 통해 각각의 정보 추출
- person_id, visit_occurrence_id : foreign_key이기에 인자로 받아 추출
- drug_exposure_id : 범위를 2000 ~ 2999 로 정해 drug_num 만의 범위 설정 > 해당 범위에서 비복원추출 진행
- drug_exposure_start_date : 약처방리스트를 date기준으로 뽑았기에 해당순서의 date 추출 후 저장
- drug_value : 해당순서의 date 뒤의 대소문자 구분없는 문자열 추출
    - 어차피 당일의 처방을 받는거기에 처방리스트가 여러개라고해도 datetime이 아닌이상 date는 같을 것임
    - 그렇기에 `re.findall(drug_date_list[0]+' :\s+([a-zA-Z /]*), drug_text')[num]`로 갖고옴
        - 아니면 여러 번의 처방을 /로 구분해서 할 수 있음 > 해당 포맷에 맞게 추출 가능
- dose_value : drug_value 뒤의 숫자 포맷을 추출후 저장
- unit_value : dose_value 뒤의 대소문자 구분 없는 문자열을 추출 후 저장
- route_value : unit_value 뒤의 대소문자 구분 없는 문자열 추출 후 저장

<br/>

- 위의 정보들을 dictionary 형태로 저장해 처방 개수만큼 drug_df에 순차적으로 저장

In [6]:
def input_drug_exposure(text, person_id, visit_id):
    # MEDICATIONS ~ CONDITIONS 사이의 텍스트만 추출
    drug_text = text[text.find('MEDICATIONS'):text.find('CONDITIONS')]
    
    drug_data = {}
    # 여러 개의 약처방이 가능하기에 try문을 통해 정보 추출 -> 약처방을 하지 않았다면 해당 note는 pass
    try:
        # 약을 처방한 date에 대한 정보 추출
        drug_date_list = re.findall('([0-9]{4}-[0-9]{2}-[0-9]{2})', drug_text)
        # 첫 번째 약처방부터 마지막 drug에 대한 정보 추출
        for num in range(len(drug_date_list)):
            # 해당 note의 person_id와 visit_occurrence_id 정보 추출
            drug_data['person_id'] = int(person_id)
            drug_data['visit_occurrence_id'] = int(visit_id)
            
            # drug_exposure_id에 대한 중복여부 검사
            drug_exposure_num = random.randint(2000, 2999)
            while drug_exposure_num in drug_df['drug_exposure_id'].values:
                drug_exposure_num = random.randint(2000, 2999)
                
            # drug_exposure_id와 drug_exposure_start_date 정보 추출
            drug_data['drug_exposure_id'] = int(drug_exposure_num)
            drug_data['drug_exposure_start_date'] = drug_date_list[num]
            
            # 해당 처방건의 drug_value 추출
            drug_name = re.findall(drug_date_list[num]+' :\s+([a-zA-z]+)', drug_text)[0]
            drug_data['drug_value'] = drug_name
            
            # 해당 처방건의 dose_value 추출
            dose = re.findall(drug_name+'\s+(\d+)', drug_text)[0]
            drug_data['dose_value'] = dose
            
            # 해당 처방건의 unit_value 추출
            unit = re.findall(dose+'\s+([a-zA-Z]+)', drug_text)[0]
            drug_data['unit_value'] = unit
            
            # 해당 처방건의 route_value 추출
            route = re.findall(unit+'\s+([a-zA-Z]+)', drug_text)[0]
            drug_data['route_value'] = route
            
            # 해당 처방건의 drug_exposure의 정보 저장
            drug_df.loc[len(drug_df)] = drug_data
    except:
        pass

#### Note에서 condition_occurrence 정보를 추출하는 함수
- input_condition_occurrence 정보를 추출할 수 있는 텍스트 범위만 추출
- 한 note 당 여러 진단이 이루어질 수도 있기에 반복문을 사용한 정보 추출
- 방문만 한 note도 있기에 예외처리를 통해 data 추출

<br/>

1. 진단의 포맷이 처음에 date 문자열부터 시작하기에 진단리스트에 0000-00-00 포맷의 문자열을 추출 후 저장
2. 각 note의 진단은 같은 날 이루어지는 것이기에 진단리스트의 첫번째 값을 뽑아 그 뒤의 `-`, ` `, 대소문자 구분 없는 텍스트를 추출해 진단명리스트를 저장
3. 진단수만큼 반복문을 통해 각 진단의 정보를 추출
<br/>

- person_id, visit_occurrence_id : foreign_key 이기에 인자로 받아 정보 저장
- condition_occurrence_id : 범위를 3000 ~ 3999 로 정해 condition 만의 범위 설정 > 해당 범위에서 비복원추출 진행
- condition_start_date : 진단리스트에서 해당 순서의 date 추출 후 저장
- condition_value : 진단명리스트에서 해당 순서의 value 추출 후 저장

<br/>

- 위의 정보들을 dictionary 형태로 받아 condition_df에 순차적으로 저장

In [7]:
def input_condition_occurrence(text, person_id, visit_id):
    # CONDITIONS ~ CARE PLANS 사이의 텍스트만 추출
    condition_text = text[text.find('CONDITIONS'):text.find('CARE PLANS')]
    
    condition_data = {}
    # 여러 개의 진단이 가능하기에 try문을 통해 정보 추출 -> 진단을 하지 않았다면 해당 note는 pass
    try:
        # 해당 note의 진단 date list 정보 추출
        condition_date_list = re.findall('([0-9]{4}-[0-9]{2}-[0-9]{2})', condition_text)
        # 해당 note의 진단명 list 정보 추출
        condition_value_list = re.findall(condition_date_list[0]+'\s+:\s+([a-zA-z \d-]+)', condition_text)
        
        # 각 진단별로 정보 추출
        for num in range(len(condition_date_list)):
            # 해당 note의 person_id와 visit_occurrence_id 정보 추출
            condition_data['person_id'] = int(person_id)
            condition_data['visit_occurrence_id'] = int(visit_id)
            
            # condition_occurrence_id에 대한 중복여부 검사
            condition_num = random.randint(3000, 3999)
            while condition_num in condition_df['condition_occurrence_id'].values:
                condition_num = random.randint(3000, 3999)
            
            # 해당 진단에 대한 condition_occurrence_id, condition_start_date, condition_value 추출
            condition_data['condition_occurrence_id'] = int(condition_num)
            condition_data['condition_start_date'] = condition_date_list[num]
            condition_data['condition_value'] = condition_value_list[num]
            
            # 해당 진단의 condition_occurrence 정보 저장
            condition_df.loc[len(condition_df)] = condition_data
    except:
        pass

#### 4가지 정보에 대한 데이터프레임 Schema 생성

In [8]:
person_df = pd.DataFrame(columns=['person_id', 'year_of_birth', 'month_of_birth', 'day_of_birth',
                                 'gender_value', 'race_value', 'ethnicity_value'])
visit_df = pd.DataFrame(columns=['visit_occurrence_id', 'person_id', 'visit_start_date',
                                'care_site_nm', 'visit_type_value'])
drug_df = pd.DataFrame(columns=['drug_exposure_id', 'person_id', 'drug_exposure_start_date',
                               'drug_value', 'route_value', 'dose_value', 'unit_value', 'visit_occurrence_id'])
condition_df = pd.DataFrame(columns=['condition_occurrence_id', 'person_id', 'condition_start_date',
                                    'condition_value', 'visit_occurrence_id'])

#### 23개의 Note에 대한 4가지의 정보 데이터 추출 후, 데이터프레임에 저장

In [9]:
for idx, note in enumerate(note_df['note']):
    print(f'Current Note Number : {idx}')
    
    # 해당 note의 CONTINUING 부분 제외
    if note.find('CONTINUING') >= 0:
        del_idx = note.find('CONTINUING')
        note = note[:del_idx]
        
    # 해당 note의 person 정보 추가 및 현재 note의 person_id 반환
    person_id = input_person(note)
    
    # 해당 note의 visit_occurrence 정보 추가 및 현재 note의 visit_occurrence_id 반환
    visit_id = input_visit_occurrence(note, person_id)
    
    # 해당 note의 drug_exposure 정보 추가
    input_drug_exposure(note, person_id, visit_id)
    
    # 해당 note의 condition_occurrence 정보 추가
    input_condition_occurrence(note, person_id, visit_id)

Current Note Number : 0
Current Note Number : 1
Current Note Number : 2
Current Note Number : 3
Current Note Number : 4
Current Note Number : 5
Current Note Number : 6
Current Note Number : 7
Current Note Number : 8
Current Note Number : 9
Current Note Number : 10
Current Note Number : 11
Current Note Number : 12
Current Note Number : 13
Current Note Number : 14
Current Note Number : 15
Current Note Number : 16
Current Note Number : 17
Current Note Number : 18
Current Note Number : 19
Current Note Number : 20
Current Note Number : 21
Current Note Number : 22


In [10]:
person_df

,person_id,year_of_birth,month_of_birth,day_of_birth,gender_value,race_value,ethnicity_value
0,19,1965,04,22,M,White,Non-Hispanic
1,1,1965,04,22,M,White,Non-Hispanic
2,8,1965,04,22,M,White,Non-Hispanic
3,10,1965,04,22,M,White,Non-Hispanic
4,9,1965,04,22,M,White,Non-Hispanic
5,14,1965,04,22,M,White,Non-Hispanic
6,6,1965,04,22,M,White,Non-Hispanic
7,11,1965,04,22,M,White,Non-Hispanic
8,7,1965,04,22,M,White,Non-Hispanic
9,15,1965,04,22,M,White,Non-Hispanic


In [11]:
visit_df

,visit_occurrence_id,person_id,visit_start_date,care_site_nm,visit_type_value
0,1498,19,2011-06-20,Cape Cod Vet Center,ambulatory
1,1432,1,2015-04-23,Cape Cod Vet Center,ambulatory
2,1606,8,2015-04-23,Cape Cod Vet Center,ambulatory
3,1729,10,2013-05-16,Cape Cod Vet Center,wellness
4,1091,9,2011-05-12,Cape Cod Vet Center,wellness
5,1550,14,1977-04-10,FALMOUTH HOSPITAL,ambulatory
6,1464,6,2017-05-04,Cape Cod Vet Center,wellness
7,1724,11,2016-04-28,Cape Cod Vet Center,wellness
8,1093,7,1996-07-04,Cape Cod Vet Center,wellness
9,1951,15,2019-05-16,Cape Cod Vet Center,wellness


In [12]:
drug_df

,drug_exposure_id,person_id,drug_exposure_start_date,drug_value,route_value,dose_value,unit_value,visit_occurrence_id
0,2185,19,2011-06-20,Acetaminophen,Oral,325,MG,1498
1,2539,20,1984-06-07,FLUoxetine,Oral,20,MG,1731


In [13]:
condition_df

,condition_occurrence_id,person_id,condition_start_date,condition_value,visit_occurrence_id
0,3786,19,2011-06-20,Acute bronchitis,1498
1,3374,1,2015-04-23,Anemia,1432
2,3603,10,2013-05-16,Prediabetes,1729
3,3687,14,1977-04-10,Chronic sinusitis,1550
4,3383,7,1996-07-04,Body mass index 30,1093
5,3752,20,1984-06-07,Major depression disorder,1731
6,3883,2,2019-04-21,Acute bacterial sinusitis,1497
7,3583,12,2014-01-26,Viral sinusitis,1600
8,3690,22,2020-02-25,COVID-19,1777
9,3636,22,2020-02-25,Suspected COVID-19,1777


---

### SQLAlchemy를 사용해 Postgres DB로 데이터 저장

#### Base 클래스 선언 (모델 생성) 

In [14]:
Base = declarative_base()

#### person 테이블에 연결할 class 생성

In [15]:
class Person(Base):
    __tablename__ = 'note_person'
    person_id = Column(BIGINT, primary_key=True)
    year_of_birth = Column(Integer, nullable=False)
    month_of_birth = Column(Integer)
    day_of_birth = Column(Integer)
    gender_value = Column(String(50))
    race_value = Column(String(50))
    ethnicity_value = Column(String(50))
    
    def __init__(self, person_id, year_of_birth, month_of_birth, day_of_birth, gender_value, race_value, ethnicity_value):
        self.person_id = person_id
        self.year_of_birth = year_of_birth
        self.month_of_birth = month_of_birth
        self.day_of_birth = day_of_birth
        self.gender_value = gender_value
        self.race_value = race_value
        self.ethnicity_value = ethnicity_value
        
    def __repr__(self):
        return f'<Person {self.person_id}, {self.year_of_birth}>'

#### visit_occurrence 테이블에 연결할 class 생성

In [16]:
class VisitOccurence(Base):
    __tablename__ = 'note_visit_occurrence'
    visit_occurrence_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    visit_start_date = Column(Date)
    care_site_nm = Column(TEXT)
    visit_type_value = Column(String(50))
    
    def __init__(self, visit_occurrence_id, person_id, visit_start_date, care_site_nm, visit_type_value):
        self.visit_occurrence_id = visit_occurrence_id
        self.person_id = person_id
        self.visit_start_date = visit_start_date
        self.care_site_nm = care_site_nm
        self.visit_type_value = visit_type_value
        
    def __repr__(self):
        return f'<VisitOccurrence {self.visit_occurrence_id}, {self.person_id}>'

#### drug_exposure 테이블에 연결할 class 생성

In [17]:
class DrugExposure(Base):
    __tablename__ = 'note_drug_exposure'
    drug_exposure_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    drug_exposure_start_date = Column(Date)
    drug_value = Column(TEXT)
    route_value = Column(String(50))
    dose_value = Column(String(50))
    unit_value = Column(String(50))
    visit_occurrence_id = Column(BIGINT, ForeignKey('note_visit_occurrence.visit_occurrence_id'))
    
    def __init__(self, drug_exposure_id, person_id, drug_exposure_start_date, drug_value, route_value, dose_value, unit_value, visit_occurrence_id):
        self.drug_exposure_id = drug_exposure_id
        self.person_id = person_id
        self.drug_exposure_start_date = drug_exposure_start_date
        self.drug_value = drug_value
        self.route_value = route_value
        self.dose_value = dose_value
        self.unit_value = unit_value
        self.visit_occurrence_id = visit_occurrence_id
        
    def __repr__(self):
        return f'<DrugExposure {self.drug_exposure_id}, {self.person_id}, {self.visit_occurrence_id}>'

#### condition_occurrence테이블에 연결할 class 생성

In [18]:
class ConditionOccurrence(Base):
    __tablename__ = 'note_condition_occurrence'
    condition_occurrence_id = Column(BIGINT, primary_key=True)
    person_id = Column(BIGINT, ForeignKey('note_person.person_id'))
    condition_start_date = Column(Date)
    condition_value = Column(TEXT)
    visit_occurrence_id = Column(BIGINT, ForeignKey('note_visit_occurrence.visit_occurrence_id'))
    
    def __init__(self, condition_occurrence_id, person_id, condition_start_date, condition_value, visit_occurrence_id):
        self.condition_occurrence_id = condition_occurrence_id
        self.person_id = person_id
        self.condition_start_date = condition_start_date
        self.condition_value = condition_value
        self.visit_occurrence_id = visit_occurrence_id
        
    def __repr__(self):
        return f'<ConditionOccurrence {self.condition_occurrence_id}, {self.person_id}, {self.visit_occurrence_id}>'

#### 선언된 Class들과 데이터베이스의 스키마 동기화

In [19]:
Base.metadata.create_all(client)

---

### DB에 저장하는 Process
- foreign_key로 테이블이 연결되있기에 person > visit > drug, condition 순으로 저장 진행
- 각 df별로 반복문으로 행을 추출해 unpacking을 통해 각 table과 연결된 객체에 insert
- 저장된 data들의 transaction을 실행 후 안전을 위해 rollback을 통해 task 초기화

<br/>

- 예외처리 문에서 finally 를 사용해 error가 발생해도 세션을 종료할 수 있게 진행
- 반복된 코드를 함수화 진행

#### 세션 생성

In [20]:
session = sessionmaker(bind=client)()

#### person_df에 저장된 데이터를 person 테이블에 Insert

In [17]:
person_datas = []
for data in person_df.iterrows():
    data = data[1].to_dict()
    person_datas.append(Person(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [18]:
session.add_all(person_datas)
session.commit()
session.rollback()

#### visit_df 저장된 데이터를 visit_occurrence 테이블에 Insert

In [19]:
visit_datas = []
for data in visit_df.iterrows():
    data = data[1].to_dict()
    visit_datas.append(VisitOccurence(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [20]:
session.add_all(visit_datas)
session.commit()
session.rollback()

#### drug_df 저장된 데이터를 drug_exposure 테이블에 Insert

In [21]:
drug_datas = []
for data in drug_df.iterrows():
    data = data[1].to_dict()
    drug_datas.append(DrugExposure(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [22]:
session.add_all(drug_datas)
session.commit()
session.rollback()

#### condition_df 저장된 데이터를 condition_occurrence 테이블에 Insert

In [23]:
condition_datas = []
for data in condition_df.iterrows():
    data = data[1].to_dict()
    condition_datas.append(ConditionOccurrence(**data))

#### transaction 실행 후 session에 있는 task 초기화

In [24]:
session.add_all(condition_datas)
session.commit()
session.rollback()

#### 세션 종료

In [21]:
session.close()